In [1]:
import json
import datetime as dt
import time
import os
import sys
import tweepy
from tweepy import OAuthHandler

In [2]:
def load_api():
    consumer_key = 'JH8SEolf31odtvkJo031Gq58D'
    consumer_secret = 'SkP4iFvFOX8BVr4lPD2wTbfr3s14ukPx7sfpGM0Hb63y88GBWJ'
    access_token = '603247879-pOSPqQeBLCYWe4oVgDDp0b9BWukJi2bpEXlfqHeK'
    access_secret = '7Q6twKqcaLYU6FicbT3L4BqbPzoj0IfF60R1MkZq5RCX8'
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    # load the twitter API via tweepy
    return tweepy.API(auth)

#Function to take the search string and return tweets related to search subject



def tweet_search(api, query, max_tweets, max_id, since_id, geocode):
     searched_tweets = []
     while len(searched_tweets) < max_tweets:
        remaining_tweets = max_tweets - len(searched_tweets)
        try:
            new_tweets = api.search(q=query, count=remaining_tweets,
                                    since_id=str(since_id),
                                    max_id=str(max_id-1))
#                                    geocode=geocode)
            print('found',len(new_tweets),'tweets')
            if not new_tweets:
                print('no tweets found')
                break
            searched_tweets.extend(new_tweets)
            max_id = new_tweets[-1].id
        except tweepy.TweepError:
            print('exception raised, waiting 15 minutes')
            print('(until:', dt.datetime.now()+dt.timedelta(minutes=15), ')')
            time.sleep(15*60)
            break # stop the loop
     return searched_tweets, max_id


def get_tweet_id(api, date='', days_ago=9, query='a'):
    if date:
        # return an ID from the start of the given day
        td = date + dt.timedelta(days=1)
        tweet_date = '{0}-{1:0>2}-{2:0>2}'.format(td.year, td.month, td.day)
        tweet = api.search(q=query, count=1, until=tweet_date)
    else:
        # return an ID from __ days ago
        td = dt.datetime.now() - dt.timedelta(days=days_ago)
        tweet_date = '{0}-{1:0>2}-{2:0>2}'.format(td.year, td.month, td.day)
        # get list of up to 10 tweets
        tweet = api.search(q=query, count=10, until=tweet_date)
        print('search limit (start/stop):',tweet[0].created_at)
        # return the id of the first tweet in the list
        return tweet[0].id
    
    
def write_tweets(tweets, filename):
    with open(filename, 'a') as f:
        for tweet in tweets:
            json.dump(tweet._json, f)
            f.write('\n')
            


def main():
    search_phrases = ['iphone8', 'iphone x', 
                     'Apple', 'new iphone',
                     'iphone', 'iphone 8']
    time_limit = 1.50                           # runtime limit in hours
    max_tweets = 100                           # number of tweets per search (will be
                                               # iterated over) - maximum is 100
    min_days_old, max_days_old = 1, 8          # search limits e.g., from 7 to 8
                                               # gives current weekday from last week,
                                               # min_days_old=0 will search from right now
    USA = '39.8,-95.583068847656,2500km'       # this geocode includes nearly all American
                                     
            
    for search_phrase in search_phrases:
        
        print('Search phrase =', search_phrase)
        
        name = search_phrase.split()[0]
        json_file_root = name + '/'  + name
        os.makedirs(os.path.dirname(json_file_root), exist_ok=True)
        read_IDs = False
        
        if max_days_old - min_days_old == 1:
            d = dt.datetime.now() - dt.timedelta(days=min_days_old)
            day = '{0}-{1:0>2}-{2:0>2}'.format(d.year, d.month, d.day)
        else:
            d1 = dt.datetime.now() - dt.timedelta(days=max_days_old-1)
            d2 = dt.datetime.now() - dt.timedelta(days=min_days_old)
            day = '{0}-{1:0>2}-{2:0>2}_to_{3}-{4:0>2}-{5:0>2}'.format(
                  d1.year, d1.month, d1.day, d2.year, d2.month, d2.day)
        json_file = json_file_root + '_' + day + '.json'
        if os.path.isfile(json_file):
            print('Appending tweets to file named: ',json_file)
            read_IDs = True
        
        # authorize and load the twitter API
        api = load_api()
        
        # set the 'starting point' ID for tweet collection
        if read_IDs:
            # open the json file and get the latest tweet ID
            with open(json_file, 'r') as f:
                lines = f.readlines()
                max_id = json.loads(lines[-1])['id']
                print('Searching from the bottom ID in file')
        else:
            # get the ID of a tweet that is min_days_old
            if min_days_old == 0:
                max_id = -1
            else:
                max_id = get_tweet_id(api, days_ago=(min_days_old-1))
        # set the smallest ID to search for
        since_id = get_tweet_id(api, days_ago=(max_days_old-1))
        print('max id (starting point) =', max_id)
        print('since id (ending point) =', since_id)
        
        
        start = dt.datetime.now()
        end = start + dt.timedelta(hours=time_limit)
        count, exitcount = 0, 0
        while dt.datetime.now() < end:
            count += 1
            print('count =',count)
            # collect tweets and update max_id
            tweets, max_id = tweet_search(api, search_phrase, max_tweets,
                                          max_id=max_id, since_id=since_id,
                                          geocode=USA)
            # write tweets to file in JSON format
            if tweets:
                write_tweets(tweets, json_file)
                exitcount = 0
            else:
                exitcount += 1
                if exitcount == 3:
                    if search_phrase == search_phrases[-1]:
                        sys.exit('Maximum number of empty tweet strings reached - exiting')
                    else:
                        print('Maximum number of empty tweet strings reached - breaking')
                        break
if __name__ == "__main__":
    main()

Search phrase = iphone8
search limit (start/stop): 2017-09-18 23:59:59
search limit (start/stop): 2017-09-11 23:59:59
max id (starting point) = 909930032122011648
since id (ending point) = 907393317062762497
count = 1
found 82 tweets
found 11 tweets
found 2 tweets
found 4 tweets
found 0 tweets
no tweets found
count = 2
found 51 tweets
found 43 tweets
found 5 tweets
found 1 tweets
count = 3
found 78 tweets
found 17 tweets
found 5 tweets
count = 4
found 84 tweets
found 10 tweets
found 3 tweets
found 2 tweets
found 1 tweets
count = 5
found 81 tweets
found 18 tweets
found 0 tweets
no tweets found
count = 6
found 72 tweets
found 14 tweets
found 7 tweets
found 5 tweets
found 1 tweets
found 1 tweets
count = 7
found 61 tweets
found 8 tweets
found 7 tweets
found 5 tweets
found 5 tweets
found 8 tweets
found 4 tweets
found 0 tweets
no tweets found
count = 8
found 84 tweets
found 12 tweets
found 3 tweets
found 0 tweets
no tweets found
count = 9
found 70 tweets
found 23 tweets
found 5 tweets
found 

found 21 tweets
found 1 tweets
count = 179
found 95 tweets
found 5 tweets
count = 180
found 100 tweets
count = 181
found 100 tweets
count = 182
found 100 tweets
count = 183
found 100 tweets
count = 184
found 74 tweets
found 11 tweets
found 5 tweets
found 5 tweets
found 1 tweets
found 0 tweets
no tweets found
count = 185
found 67 tweets
found 24 tweets
found 5 tweets
found 0 tweets
no tweets found
count = 186
found 44 tweets
found 34 tweets
found 13 tweets
found 7 tweets
found 1 tweets
found 1 tweets
count = 187
found 76 tweets
found 20 tweets
found 2 tweets
found 0 tweets
no tweets found
count = 188
found 59 tweets
found 12 tweets
found 15 tweets
found 8 tweets
found 2 tweets
found 3 tweets
found 0 tweets
no tweets found
count = 189
found 74 tweets
found 14 tweets
found 7 tweets
found 4 tweets
found 0 tweets
no tweets found
count = 190
found 76 tweets
found 20 tweets
found 4 tweets
count = 191
found 62 tweets
found 13 tweets
found 14 tweets
found 6 tweets
found 4 tweets
found 1 tweets


found 1 tweets
count = 311
found 100 tweets
count = 312
found 100 tweets
count = 313
found 92 tweets
found 8 tweets
count = 314
found 100 tweets
count = 315
found 100 tweets
count = 316
found 99 tweets
found 1 tweets
count = 317
found 95 tweets
found 4 tweets
found 1 tweets
count = 318
found 100 tweets
count = 319
found 90 tweets
found 8 tweets
found 2 tweets
count = 320
found 97 tweets
found 2 tweets
found 1 tweets
count = 321
found 100 tweets
count = 322
found 96 tweets
found 3 tweets
found 0 tweets
no tweets found
count = 323
found 85 tweets
found 11 tweets
found 1 tweets
found 1 tweets
found 1 tweets
found 1 tweets
count = 324
found 97 tweets
found 3 tweets
count = 325
found 97 tweets
found 2 tweets
found 0 tweets
no tweets found
count = 326
found 95 tweets
found 5 tweets
count = 327
found 99 tweets
found 1 tweets
count = 328
found 100 tweets
count = 329
found 100 tweets
count = 330
found 94 tweets
found 6 tweets
count = 331
found 93 tweets
found 6 tweets
found 0 tweets
no tweets f

found 100 tweets
count = 176
exception raised, waiting 15 minutes
(until: 2017-09-19 18:36:26.830112 )
Search phrase = Apple
search limit (start/stop): 2017-09-18 23:59:59
search limit (start/stop): 2017-09-11 23:59:59
max id (starting point) = 909930032122011648
since id (ending point) = 907393317062762497
count = 1
found 100 tweets
count = 2
found 100 tweets
count = 3
found 100 tweets
count = 4
found 100 tweets
count = 5
found 100 tweets
count = 6
found 100 tweets
count = 7
found 100 tweets
count = 8
found 100 tweets
count = 9
found 100 tweets
count = 10
found 100 tweets
count = 11
found 100 tweets
count = 12
found 100 tweets
count = 13
found 100 tweets
count = 14
found 100 tweets
count = 15
found 100 tweets
count = 16
found 100 tweets
count = 17
found 100 tweets
count = 18
found 100 tweets
count = 19
found 100 tweets
count = 20
found 100 tweets
count = 21
found 100 tweets
count = 22
found 100 tweets
count = 23
found 100 tweets
count = 24
found 100 tweets
count = 25
found 100 tweets


found 100 tweets
count = 273
found 100 tweets
count = 274
found 100 tweets
count = 275
found 100 tweets
count = 276
found 76 tweets
found 24 tweets
count = 277
found 100 tweets
count = 278
found 100 tweets
count = 279
found 100 tweets
count = 280
found 100 tweets
count = 281
found 100 tweets
count = 282
found 100 tweets
count = 283
found 100 tweets
count = 284
found 100 tweets
count = 285
found 100 tweets
count = 286
found 100 tweets
count = 287
found 100 tweets
count = 288
found 100 tweets
count = 289
found 100 tweets
count = 290
found 100 tweets
count = 291
found 100 tweets
count = 292
found 100 tweets
count = 293
found 100 tweets
count = 294
found 100 tweets
count = 295
found 100 tweets
count = 296
found 100 tweets
count = 297
found 100 tweets
count = 298
found 100 tweets
count = 299
found 100 tweets
count = 300
found 100 tweets
count = 301
found 100 tweets
count = 302
found 100 tweets
count = 303
found 100 tweets
count = 304
found 100 tweets
count = 305
found 100 tweets
count = 306

count = 89
found 100 tweets
count = 90
found 100 tweets
count = 91
found 100 tweets
count = 92
found 100 tweets
count = 93
found 89 tweets
found 11 tweets
count = 94
found 100 tweets
count = 95
found 100 tweets
count = 96
found 100 tweets
count = 97
found 100 tweets
count = 98
found 100 tweets
count = 99
found 100 tweets
count = 100
found 100 tweets
count = 101
found 100 tweets
count = 102
found 100 tweets
count = 103
found 100 tweets
count = 104
found 100 tweets
count = 105
found 100 tweets
count = 106
found 100 tweets
count = 107
found 100 tweets
count = 108
found 100 tweets
count = 109
found 100 tweets
count = 110
found 100 tweets
count = 111
found 100 tweets
count = 112
found 100 tweets
count = 113
found 100 tweets
count = 114
found 100 tweets
count = 115
found 100 tweets
count = 116
found 100 tweets
count = 117
found 100 tweets
count = 118
found 100 tweets
count = 119
found 100 tweets
count = 120
found 100 tweets
count = 121
found 100 tweets
count = 122
found 100 tweets
count = 12

found 100 tweets
count = 357
found 100 tweets
count = 358
found 100 tweets
count = 359
found 100 tweets
count = 360
found 100 tweets
count = 361
found 100 tweets
count = 362
found 100 tweets
count = 363
found 37 tweets
found 63 tweets
count = 364
found 100 tweets
count = 365
found 95 tweets
found 5 tweets
count = 366
found 100 tweets
count = 367
found 100 tweets
count = 368
found 100 tweets
count = 369
found 99 tweets
found 1 tweets
count = 370
found 100 tweets
count = 371
found 100 tweets
count = 372
found 100 tweets
count = 373
found 100 tweets
count = 374
found 100 tweets
count = 375
found 100 tweets
count = 376
found 100 tweets
count = 377
found 100 tweets
count = 378
found 100 tweets
count = 379
found 100 tweets
count = 380
found 100 tweets
count = 381
found 100 tweets
count = 382
found 100 tweets
count = 383
found 100 tweets
count = 384
found 100 tweets
count = 385
found 100 tweets
count = 386
found 100 tweets
count = 387
found 100 tweets
count = 388
found 100 tweets
count = 389


found 100 tweets
count = 621
found 100 tweets
count = 622
found 100 tweets
count = 623
found 100 tweets
count = 624
found 100 tweets
count = 625
found 100 tweets
count = 626
found 100 tweets
count = 627
found 100 tweets
count = 628
found 100 tweets
count = 629
found 100 tweets
count = 630
found 100 tweets
count = 631
found 100 tweets
count = 632
found 100 tweets
count = 633
found 100 tweets
count = 634
found 100 tweets
count = 635
found 100 tweets
count = 636
found 100 tweets
count = 637
found 100 tweets
count = 638
found 100 tweets
count = 639
found 100 tweets
count = 640
found 100 tweets
count = 641
found 100 tweets
count = 642
found 98 tweets
found 2 tweets
count = 643
found 100 tweets
count = 644
found 100 tweets
count = 645
found 100 tweets
count = 646
found 100 tweets
count = 647
found 100 tweets
count = 648
found 100 tweets
count = 649
found 100 tweets
count = 650
found 100 tweets
count = 651
found 100 tweets
count = 652
found 100 tweets
count = 653
found 100 tweets
count = 654


count = 227
found 100 tweets
count = 228
found 100 tweets
count = 229
found 100 tweets
count = 230
found 100 tweets
count = 231
found 100 tweets
count = 232
found 100 tweets
count = 233
found 100 tweets
count = 234
found 100 tweets
count = 235
found 100 tweets
count = 236
found 100 tweets
count = 237
found 100 tweets
count = 238
found 100 tweets
count = 239
found 100 tweets
count = 240
found 100 tweets
count = 241
found 100 tweets
count = 242
found 100 tweets
count = 243
found 100 tweets
count = 244
found 100 tweets
count = 245
found 100 tweets
count = 246
found 100 tweets
count = 247
found 100 tweets
count = 248
found 100 tweets
count = 249
found 100 tweets
count = 250
found 100 tweets
count = 251
found 100 tweets
count = 252
found 100 tweets
count = 253
found 100 tweets
count = 254
found 100 tweets
count = 255
found 100 tweets
count = 256
found 100 tweets
count = 257
found 100 tweets
count = 258
found 100 tweets
count = 259
found 100 tweets
count = 260
found 100 tweets
count = 261
fo

KeyboardInterrupt: 

In [ ]:
import json
 
tweet_files = ['file_1.json', 'file_2.json', ...]
tweets = []
for file in tweet_files:
    with open(file, 'r') as f:
        for line in f.readlines():
            tweets.append(json.loads(line))